In [1]:
import Pkg
 
package_list = ["DataFrames", "CSV",
  "JuMP", "Gurobi",
  "LinearAlgebra", "Random", "Printf", "StatsBase", "CategoricalArrays",
  "Plots", "StatsPlots",
  "Distributions", "JLD"]

for pkg_name in package_list
    Pkg.add(pkg_name)
end

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

In [2]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [3]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


Gurobi.Env(Ptr{Nothing} @0x00007fb31d5c6a00, false, 0)

# Problem setup

30 years later, Filatoi is doing great! Milan Consulting Group Inc. converted them to optimization, and have kept using it throughout their expansions.

They have decided to expand the range of yarn they produce by including colored products. They still produce $I=4$ types of yarns: extra fine yarn, fine yarn, medium yarn and coarse yarn but with $K = 16$ different color tones: red, orange, yellow, green, blue, indigo, violet, purple, pink, silver, gold, beige, brown, gray, black, and white.

Since they have now expanded over all of Italy, they have many more outsourcing options. They have 99 mills to outsource to, on top of their inhouse production - for a total of $J=100$ mills.

Filatoi has demand $d_{i,k}$ for each type of yarn $i$ and color $k$.

They also have maximum work hours at each of the $j$ mills $h_{j}$.

There are machine hour requirements for producing yarn $i$ of color $k$ at mill $j$: $r_{i,j,k}$ hours/kg.

There are costs for producing yarn $i$ of color $k$ at mill $j$: $c_{i,j,kl}$ \$/kg (this includes transportation).

## The data

In [26]:
T = 52
i = 100
M = 500

500

In [65]:
A = DataFrame(CSV.File("availability.csv"))
A = select!(A, Not(:material));

In [66]:
R = DataFrame(CSV.File("requirements.csv"))
R = select!(R, Not(:materials));

In [67]:
D = DataFrame(CSV.File("demand.csv"))
D = select!(D, Not(:product));

In [68]:
P = DataFrame(CSV.File("profit.csv"))
P = select!(P, Not(:product));

In [69]:
#data_file = load("data.jld")
H = DataFrame(CSV.File("holding.csv"))
H = select!(H, Not(:week));
#c, r, d, h = data_file["costs"], data_file["hours_requirements"], data_file["demand"], data_file["machine_hours"];

In [ ]:
#colors = ["red", "orange", "yellow", "green", "blue",
#    "indigo", "violet", "purple", "pink", "silver",
#    "gold", "beige", "brown", "gray", "black", "white"]

#yarn_type = ["extra fine", "fine", "medium", "coarse"];

In [ ]:
#function print_planning_mill_j(x, j)
#    df = DataFrame( transpose(x[:,j,:]), yarn_type)
#    insertcols!(df, 1, "Color" => colors)
#    show(df)
#end
#;

# 1.) A naïve approach

Let's first design a baseline method to satisfy demand.

One way we can approach this is to try to satisfy demand for all yarns in all colors one by one. For each item, we will choose the most cost-effective (i.e. the least \$/kg), then the second most cost-effective, etc...

Does this necessarily yield a feasible strategy? Not necessarily, but we can try!

In [ ]:
x_naive = zeros((I,J,K))

working_hours_left = copy(h)
demand_left = copy(d)

for i=1:I, k=1:K
    for j in sortperm(c[i,:,k] ./ r[i,:,k])
        if demand_left[i,k] == 0
            break
        elseif demand_left[i,k] > 0 && working_hours_left[j] > 0
            x_max = min(working_hours_left[j] / r[i,j,k], demand_left[i,k])
            x_naive[i,j,k] = x_naive[i,j,k] + x_max
            working_hours_left[j] = working_hours_left[j] - r[i,j,k] * x_max
            demand_left[i,k] = demand_left[i,k] - x_max
        end
    end
end

In [ ]:
obj_naive = sum(c[i,j,k] * x_naive[i,j,k] for i=1:I, j=1:J, k=1:K)

# 2.) An optimal approach

## Formulating the problem

We want to solve the following problem:

$$\begin{align*}
                \min_{\mathbf{x}}		&	\quad	\sum_{i=1}^{I}\sum_{j=1}^{J}\sum_{k=1}^{K}c_{i,j,k}x_{i,j,k}				\\
                \text{s.t.}	&	\quad	\sum_{j=1}^{J}x_{i,j,k}\geq d_{i,k} \quad \forall i \in [I], \, \forall 	k \in [K]\\
                		    &	\quad	\sum_{i=1}^{I}\sum_{k=1}^{K} r_{i,j,k}x_{i,j,k}\leq h_{j} \quad \forall j\in[J]	\\
                		    &	\quad	\mathbf{x} \geq 0
            \end{align*}$$

where:
$$\begin{aligned}
    x_{i,j,k}&:\ \text{amount of yarn $i$ of color $k$ manufactured at mill $j$ (in kg)}
\end{aligned}$$

## Creating the model

In [16]:
model = Model(Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

## Variables

In [76]:
unregister(model, :X)
@variable(model, X[1:i, 1:T] >= 0);
unregister(model, :C)
@variable(model, C[1:i] >= 0);
unregister(model, :H)
@variable(model, H[1:T] >= 0);
unregister(model, :H)
@variable(model, B[1:i, 1:T] >= 0);

## Objective

In [54]:
@objective(model, Min, sum(C .* X) )

C[1]*X[1,1] + C[2]*X[2,1] + C[3]*X[3,1] + C[4]*X[4,1] + C[5]*X[5,1] + C[6]*X[6,1] + C[7]*X[7,1] + C[8]*X[8,1] + C[9]*X[9,1] + C[10]*X[10,1] + C[11]*X[11,1] + C[12]*X[12,1] + C[13]*X[13,1] + C[14]*X[14,1] + C[15]*X[15,1] + C[16]*X[16,1] + C[17]*X[17,1] + C[18]*X[18,1] + C[19]*X[19,1] + C[20]*X[20,1] + C[21]*X[21,1] + C[22]*X[22,1] + C[23]*X[23,1] + C[24]*X[24,1] + C[25]*X[25,1] + C[26]*X[26,1] + C[27]*X[27,1] + C[28]*X[28,1] + C[29]*X[29,1] + C[30]*X[30,1] + C[31]*X[31,1] + C[32]*X[32,1] + C[33]*X[33,1] + C[34]*X[34,1] + C[35]*X[35,1] + C[36]*X[36,1] + C[37]*X[37,1] + C[38]*X[38,1] + C[39]*X[39,1] + C[40]*X[40,1] + C[41]*X[41,1] + C[42]*X[42,1] + C[43]*X[43,1] + C[44]*X[44,1] + C[45]*X[45,1] + C[46]*X[46,1] + C[47]*X[47,1] + C[48]*X[48,1] + C[49]*X[49,1] + C[50]*X[50,1] + C[51]*X[51,1] + C[52]*X[52,1] + C[53]*X[53,1] + C[54]*X[54,1] + C[55]*X[55,1] + C[56]*X[56,1] + C[57]*X[57,1] + C[58]*X[58,1] + C[59]*X[59,1] + C[60]*X[60,1] + C[61]*X[61,1] + C[62]*X[62,1] + C[63]*X[63,1] + C[64]*X[64

## Constraints

### Demand constraints

In [74]:
unregister(model, :resource_constraint)
@constraint(model, resource_constraint[m in 1:M, t in 1:T], sum(R[m,i]*X[i,t] for item=1:i) <= A[m,t])


500×52 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 resource_constraint[1,1] : 0 ≤ 70268.44                              …  resource_constraint[1,52] : 0 ≤ 78106.76
 resource_constraint[2,1] : 0 ≤ 5280.24                                  resource_constraint[2,52] : 0 ≤ 7230.42
 resource_constraint[3,1] : 0 ≤ 122626.49                                resource_constraint[3,52] : 0 ≤ 50871.77
 resource_constraint[4,1] : 0 ≤ 41903.88                                 resource_constraint[4,52] : 0 ≤ 36162.39
 resource_constraint[5,1] : 0 ≤ 65296.83                                 resource_constraint[5,52] : 0 ≤ 41212.57
 resource_constraint[6,1] : 4032.0000000000073 X[100,1] ≤ 35631.12    …  resource_constraint[6,52] : 4032.0000000000073 X[100,52] ≤ 59549.14
 resource_constraint[7,1] : 0 ≤ 17404.27                                 resource_constraint[7,52] : 0 ≤ 80155.89
 resourc

In [82]:
unregister(model, :demand_constraint)
@constraint(model, demand_constraint[item=1:i, t in 1:T], sum(X[i,t]+B[i,t]) >= D[i,t])

100×52 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 demand_constraint[1,1] : X[100,1] + B[100,1] ≥ 207.31    …  demand_constraint[1,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_constraint[2,1] : X[100,1] + B[100,1] ≥ 207.31       demand_constraint[2,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_constraint[3,1] : X[100,1] + B[100,1] ≥ 207.31       demand_constraint[3,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_constraint[4,1] : X[100,1] + B[100,1] ≥ 207.31       demand_constraint[4,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_constraint[5,1] : X[100,1] + B[100,1] ≥ 207.31       demand_constraint[5,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_constraint[6,1] : X[100,1] + B[100,1] ≥ 207.31    …  demand_constraint[6,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_constraint[7,1] : X[100,1] + B[100,1] ≥ 207.31       demand_constraint[7,52] : X[100,52] + B[100,52] ≥ 141.8
 demand_co

### Work hours constraints

In [ ]:
@constraint(model, work_hours_constraints[j=1:J], sum(r[i,j,k] * x[i,j,k] for i=1:I, k=1:K) <= h[j]);

## Optimizing the model

In [ ]:
optimize!(model)

## Retrieving the results

In [ ]:
x_opt = value.(x);

In [ ]:
obj_opt = objective_value(model)

# Analysis of results

## Edge of optimization

In [ ]:
edge = (obj_naive - obj_opt) / obj_naive

## Let's delve into the strategy of the optimization!

What is produced in mill 1?

In [ ]:
print_planning_mill_j(x_opt, 1)

What is produced in mill 13?

In [ ]:
print_planning_mill_j(x_opt, 13)

What is produced in mill 29?

In [ ]:
print_planning_mill_j(x_opt, 29)

What is produced in mill 57?

In [ ]:
print_planning_mill_j(x_opt, 57)

# Sensitivity analysis

## A new client asks for 1,000kg of extra fine brown wool... What price should you quote?

### Reoptimizing

In [ ]:
new_d = copy(d)
new_d[1,13] += 1000 # change the demand of brown wool

In [ ]:
model2 = Model(() -> Gurobi.Optimizer(GRB_ENV))

@variable(model2, x2[1:I,1:J,1:K] >= 0)

@objective(model2, Min, sum( c .* x2))

@constraint(model2, work_hours_constraints[j=1:J], sum(r[i,j,k] * x2[i,j,k] for i=1:I, k=1:K) <= h[j]);
@constraint(model2, demand_constraint[i in 1:I, k in 1:K], sum(x2[i,j,k] for j=1:J) >= d[i,k]);
optimize!(model2)

In [ ]:
x_opt2 = value.(x2);

### A smarter approach!

Let's look at who produces extra fine brown wool:

In [ ]:
i = 1 # extra fine yarn
k = 13 # brown color

brown_extra_fine_yarn_producer = []

for j=1:J
    if x_opt[i,j,k] > 0
        println("Mill ", j, " produces ", x_opt2[i,j,k], " kg of extra fine brown yarn")
        push!(brown_extra_fine_yarn_producer, j)
    end
end

Now let's look if they have any leeway to produce more extra fine brown yarn i.e., if their machine hours constraint are non binding:

In [ ]:
for j in brown_extra_fine_yarn_producer
    if h[j] - sum(r[i,j,k]*x_opt[i,j,k] for i = 1:I, k = 1:K) == 0
        println("Mill ", j, " has a binding constraint for machine hours")
    else
        println("Mill ", j, " has a non binding constraint for machine hours - they have ", 
        h[j] - sum(r[i,j,k]*x_opt[i,j,k] for i = 1:I, k = 1:K), " hours available")
    end
end

Mill 1 still has some available machine hours! Are these work hours enough to produce the additional brown extra fine yarn?

In [ ]:
j = 1

(h[j] - sum(r[i,j,k]*x_opt[i,j,k] for i = 1:I, k = 1:K)) / r[i,j,k]

Yes! Since $14,814 \geq 1000$.

We can thus quote as a minimal price the cost it takes for mill $1$ to produce this yarn:

In [ ]:
1000 * c[i,j,k]

## Mill 1 can buy an upgrade to expand its machine hours - how much should you be willing to pay?

\$0! Mill 1 doesn't use all its current machine hours, so no need to get any more!